In [126]:
# TEMPLATE: https://github.com/OpenSourceBrain/CalciumImagingDriftingGrating/blob/master/Convert.py
# SAMPLE NOTEBOOK: https://github.com/NeurodataWithoutBorders/nwb_tutorial/blob/master/HCK08/ophys_tutorial.ipynb

import glob, os
from datetime import datetime
from dateutil.tz import tzlocal
import numpy as np
import pynwb
from pynwb import NWBFile, TimeSeries, NWBHDF5IO
from pynwb.image import ImageSeries
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, \
    Fluorescence, CorrectedImageStack, MotionCorrection, RoiResponseSeries


#REQUIRED: session_id, experiment_description, session_start_time

description = 'mouse exploring in open field'
identifier = 'Mouse5_Day3'
session_start_time = datetime.utcnow()

#optional below
session_id = 'session_1234'
experimenter = 'Arash'
institution='University of California - San Diego'
doi_pub='DOI:10/1016/j.neuron'
lab_performance_center='David Kleinfeld'

nwbfile = NWBFile(
    identifier=identifier,
    session_description=description,
    session_start_time=session_start_time,
    session_id=session_id,
    experimenter=experimenter,
    institution=institution,
    related_publications=doi_pub,
    lab=lab_performance_center,
)
print(nwbfile)

root pynwb.file.NWBFile at 0x139708104185504
Fields:
  experimenter: ['Arash']
  file_create_date: [datetime.datetime(2022, 8, 15, 12, 59, 51, 75038, tzinfo=tzlocal())]
  identifier: Mouse5_Day3
  institution: University of California - San Diego
  lab: David Kleinfeld
  related_publications: ['DOI:10/1016/j.neuron']
  session_description: mouse exploring in open field
  session_id: session_1234
  session_start_time: 2022-08-15 19:59:51.074485-07:00
  timestamps_reference_time: 2022-08-15 19:59:51.074485-07:00



In [130]:
#OPTIONAL
from pynwb.file import Subject

nwbfile.subject = Subject(
    subject_id='001',
    age='P90D',
    description='mouse 5',
    species='Mus musculus',
    sex='M'
)

In [ ]:
#OPTIONAL
from pynwb.behavior import SpatialSeries
from pynwb.behavior import Position


# create fake data with shape (50, 2)
# the first dimension should always represent time
position_data = np.array([np.linspace(0, 10, 50),
                          np.linspace(0, 8, 50)]).T
position_timestamps = np.linspace(0, 50) / 200

spatial_series_obj = SpatialSeries(
    name='SpatialSeries',
    description='(x,y) position in open field',
    data=position_data,
    timestamps=position_timestamps,
    reference_frame='(0,0) is bottom left corner'
)


position_obj = Position(spatial_series=spatial_series_obj)

In [127]:
ext_files_path = u"/mnt/e/temp/rui/"

image_series_external_files = {}
for root, dirs, files in os.walk(ext_files_path):
    path = root.split(os.sep)
    folder_name = os.path.basename(root)

    image_series = []
    for file in files:
        file_name = file
        c_time = os.path.getctime(os.path.join(ext_files_path, folder_name, file))
        dt_creation_time = datetime.fromtimestamp(c_time)
        creation_time = datetime.fromtimestamp(c_time)
        #creation_time = dt_creation_time.strftime("%Y-%b-%d %H:%M:%S")
        image_series.append([os.path.join(folder_name, file_name), creation_time])
    if len(folder_name) > 0:
        image_series_external_files[folder_name] = image_series


In [128]:
image_series = []
for key, files in image_series_external_files.items():
    description = f'{key}'
    timestamp = files[1]
    comments = f'FILES {len(files)}'

    ext_files = []
    ext_files_timestamps = []
    for filename in files:
        ext_files.append(files[0][0])
        #ext_files_timestamps.append(files[0][1])
        dt = datetime(2018, 10, 22, 0, 0)

        timestamp = 1563907835.857213
        ext_files_timestamps = np.arange(len(files)) + timestamp



    image_series.append(ImageSeries(name=f'image_series_{description}',
                                   external_file=ext_files,
                                   starting_frame=[0],
                                   timestamps=ext_files_timestamps,
                                   format='external',
                                   description=description,
                                   comments=comments))
for series in image_series:
    nwbfile.add_acquisition(series)

image_series

[image_series_20210309_WT_VPM_556.857_2 pynwb.image.ImageSeries at 0x139708104184448
 Fields:
   comments: FILES 60
   conversion: 1.0
   description: 20210309_WT_VPM_556.857_2
   external_file: ['20210309_WT_VPM_556.857_2/335um_roi1_10Hz_00001.tif'
  '20210309_WT_VPM_556.857_2/335um_roi1_10Hz_00001.tif'
  '20210309_WT_VPM_556.857_2/335um_roi1_10Hz_00001.tif'
  '20210309_WT_VPM_556.857_2/335um_roi1_10Hz_00001.tif'
  '20210309_WT_VPM_556.857_2/335um_roi1_10Hz_00001.tif'
  '20210309_WT_VPM_556.857_2/335um_roi1_10Hz_00001.tif'
  '20210309_WT_VPM_556.857_2/335um_roi1_10Hz_00001.tif'
  '20210309_WT_VPM_556.857_2/335um_roi1_10Hz_00001.tif'
  '20210309_WT_VPM_556.857_2/335um_roi1_10Hz_00001.tif'
  '20210309_WT_VPM_556.857_2/335um_roi1_10Hz_00001.tif'
  '20210309_WT_VPM_556.857_2/335um_roi1_10Hz_00001.tif'
  '20210309_WT_VPM_556.857_2/335um_roi1_10Hz_00001.tif'
  '20210309_WT_VPM_556.857_2/335um_roi1_10Hz_00001.tif'
  '20210309_WT_VPM_556.857_2/335um_roi1_10Hz_00001.tif'
  '20210309_WT_VPM_556

In [129]:
nwb_path = f"./{session_id}.nwb"


nwb_file_name = nwb_path
with NWBHDF5IO(nwb_file_name, 'w') as io:
    io.write(nwbfile)

print(f"Written NWB file to {nwb_file_name}")

Written NWB file to ./session_1234.nwb
